# 1️⃣ Training an Adapter for a Transformer model

This script is taken from: https://adapterhub.ml. 


In this notebook, we train an adapter for a **RoBERTa** ([Liu et al., 2019](https://arxiv.org/pdf/1907.11692.pdf)) model for sequence classification on a **sentiment analysis** task using [adapter-transformers](https://github.com/Adapter-Hub/adapter-transformers), the _AdapterHub_ adaptation of HuggingFace's _transformers_ library.
 We download the dataset via HuggingFace's [datasets](https://github.com/huggingface/datasets) library.

## Installation

First, let's install the required libraries:

In [1]:
!pip install -U adapter-transformers
!pip install datasets

     |████████████████████████████████| 2.5 MB 7.4 MB/s 
     |████████████████████████████████| 3.3 MB 61.3 MB/s 
     |████████████████████████████████| 895 kB 62.8 MB/s 
     |████████████████████████████████| 43 kB 2.7 MB/s 
     |████████████████████████████████| 264 kB 8.5 MB/s 
     |████████████████████████████████| 118 kB 70.4 MB/s 
     |████████████████████████████████| 76 kB 5.0 MB/s 
     |████████████████████████████████| 243 kB 65.7 MB/s 
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1


In [2]:
import torch
torch.cuda.is_available()

True

In [3]:
from google.colab import drive
drive.mount("/content/gdrive/", force_remount=True)

Mounted at /content/gdrive/


In [4]:
import sys
sys.path.append('/content/gdrive/MyDrive/master_hpi/NLP_Project/code/')

In [5]:
path = "/content/gdrive/MyDrive/master_hpi/NLP_Project/code/"


In [6]:
path


'/content/gdrive/MyDrive/master_hpi/NLP_Project/code/'

## Dataset Preprocessing

Before we start to train our adapter, we first prepare the training data. Our training dataset can be loaded via HuggingFace `datasets` using one line of code:

In [7]:
from datasets import load_dataset

dataset = load_dataset("rotten_tomatoes")
dataset.num_rows

Downloading:   0%|          | 0.00/1.91k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/921 [00:00<?, ?B/s]

Using custom data configuration default


Downloading:   0%|          | 0.00/488k [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset rotten_tomatoes_movie_review downloaded and prepared to /root/.cache/huggingface/datasets/rotten_tomatoes_movie_review/default/1.0.0/e06abb624abab47e1a64608fdfe65a913f5a68c66118408032644a3285208fb5. Subsequent calls will reuse this data.


{'test': 1066, 'train': 8530, 'validation': 1066}

Every dataset sample has an input text and a binary label:

In [8]:
dataset['train'][0]

{'label': 1,
 'text': 'the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .'}

Now, we need to encode all dataset samples to valid inputs for our Transformer model. Since we want to train on `roberta-base`, we load the corresponding `RobertaTokenizer`. Using `dataset.map()`, we can pass the full dataset through the tokenizer in batches:

In [9]:
from transformers import RobertaTokenizer

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

def encode_batch(batch):
  """Encodes a batch of input data using the model tokenizer."""
  return tokenizer(batch["text"], max_length=80, truncation=True, padding="max_length")

# Encode the input data
dataset = dataset.map(encode_batch, batched=True)
# The transformers model expects the target class column to be named "labels"
dataset.rename_column_("label", "labels")
# Transform to pytorch tensors and only output the required columns
dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

  0%|          | 0/9 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: FutureWarning: rename_column_ is deprecated and will be removed in the next major version of datasets. Use DatasetDict.rename_column instead.
  if sys.path[0] == '':


Now we're ready to train our model...

## Training

We use a pre-trained RoBERTa model from HuggingFace. We use `RobertaModelWithHeads`, a class unique to `adapter-transformers`, which allows us to add and configure prediction heads in a flexibler way.

In [10]:
from transformers import RobertaConfig, RobertaModelWithHeads

config = RobertaConfig.from_pretrained(
    "roberta-base",
    num_labels=2,
)
model = RobertaModelWithHeads.from_pretrained(
    "roberta-base",
    config=config,
)

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModelWithHeads: ['lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModelWithHeads from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModelWithHeads were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.embeddings.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infere

**Here comes the important part!**

We add a new adapter to our model by calling `add_adapter()`. We pass a name (`"rotten_tomatoes"`) and [the type of adapter](https://docs.adapterhub.ml/adapters.html#adapter-types) (task adapter). Next, we add a binary classification head. It's convenient to give the prediction head the same name as the adapter. This allows us to activate both together in the next step. The `train_adapter()` method does two things:

1. It freezes all weights of the pre-trained model so only the adapter weights are updated during training.
2. It activates the adapter and the prediction head such that both are used in every forward pass.

In [11]:
# Add a new adapter
model.add_adapter("rotten_tomatoes")
# Add a matching classification head
model.add_classification_head(
    "rotten_tomatoes",
    num_labels=2,
    id2label={ 0: "👎", 1: "👍"}
  )
# Activate the adapter
model.train_adapter("rotten_tomatoes")

For training, we make use of the `Trainer` class built-in into `transformers`. We configure the training process using a `TrainingArguments` object and define a method that will calculate the evaluation accuracy in the end. We pass both, together with the training and validation split of our dataset, to the trainer instance.

**Note the differences in hyperparameters compared to full finetuning.** Adapter training usually required a few more training epochs than full finetuning.

In [12]:
import numpy as np
from transformers import TrainingArguments, Trainer, EvalPrediction

training_args = TrainingArguments(
    learning_rate=1e-4,
    num_train_epochs=6,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    logging_steps=200,
    output_dir="./training_output",
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
)

def compute_accuracy(p: EvalPrediction):
  preds = np.argmax(p.predictions, axis=1)
  return {"acc": (preds == p.label_ids).mean()}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    compute_metrics=compute_accuracy,
)

Start the training 🚀

In [13]:
trainer.train()

***** Running training *****
  Num examples = 8530
  Num Epochs = 6
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 1602


Step,Training Loss
200,0.497900
400,0.310900
600,0.285200
800,0.275800
1000,0.246500
1200,0.241700
1400,0.232600
1600,0.231300


Saving model checkpoint to ./training_output/checkpoint-500
Configuration saved in ./training_output/checkpoint-500/rotten_tomatoes/adapter_config.json
Module weights saved in ./training_output/checkpoint-500/rotten_tomatoes/pytorch_adapter.bin
Configuration saved in ./training_output/checkpoint-500/rotten_tomatoes/head_config.json
Module weights saved in ./training_output/checkpoint-500/rotten_tomatoes/pytorch_model_head.bin
Configuration saved in ./training_output/checkpoint-500/rotten_tomatoes/head_config.json
Module weights saved in ./training_output/checkpoint-500/rotten_tomatoes/pytorch_model_head.bin
Saving model checkpoint to ./training_output/checkpoint-1000
Configuration saved in ./training_output/checkpoint-1000/rotten_tomatoes/adapter_config.json
Module weights saved in ./training_output/checkpoint-1000/rotten_tomatoes/pytorch_adapter.bin
Configuration saved in ./training_output/checkpoint-1000/rotten_tomatoes/head_config.json
Module weights saved in ./training_output/check

TrainOutput(global_step=1602, training_loss=0.2900677046897855, metrics={'train_runtime': 486.7781, 'train_samples_per_second': 105.14, 'train_steps_per_second': 3.291, 'total_flos': 3098616289056000.0, 'train_loss': 0.2900677046897855, 'epoch': 6.0})

Looks good! Let's evaluate our adapter on the validation split of the dataset to see how well it learned:

In [14]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 1066
  Batch size = 32


{'epoch': 6.0,
 'eval_acc': 0.8958724202626641,
 'eval_loss': 0.2769802510738373,
 'eval_runtime': 5.4154,
 'eval_samples_per_second': 196.846,
 'eval_steps_per_second': 6.278}

We can put our trained model into a `transformers` pipeline to be able to make new predictions conveniently:

In [15]:
 from transformers import TextClassificationPipeline

classifier = TextClassificationPipeline(model=model, tokenizer=tokenizer, device=training_args.device.index)

classifier("This is awesome!")

[{'label': '👍', 'score': 0.9901895523071289}]

At last, we can also extract the adapter from our model and separately save it for later reuse. Note the size difference compared to a full model!

In [16]:
model.save_adapter(path +"models/roberta/", "rotten_tomatoes")
print(model.active_adapters)
#model.save_pretrained(path +"models/sst/", "sst-2")

!ls -lh final_adapter

Configuration saved in /content/gdrive/MyDrive/master_hpi/NLP_Project/code/models/roberta/adapter_config.json
Module weights saved in /content/gdrive/MyDrive/master_hpi/NLP_Project/code/models/roberta/pytorch_adapter.bin
Configuration saved in /content/gdrive/MyDrive/master_hpi/NLP_Project/code/models/roberta/head_config.json
Module weights saved in /content/gdrive/MyDrive/master_hpi/NLP_Project/code/models/roberta/pytorch_model_head.bin


Stack[rotten_tomatoes]
ls: cannot access 'final_adapter': No such file or directory


**Share your work!**

The next step after training is to share our adapter with the world via _AdapterHub_. [Read our guide](https://docs.adapterhub.ml/contributing.html) on how to prepare the adapter module we just saved and contribute it to the Hub!

➡️ Also continue with [the next Colab notebook](https://colab.research.google.com/github/Adapter-Hub/adapter-transformers/blob/master/notebooks/02_Adapter_Inference.ipynb) to learn how to use adapters from the Hub.